In [1]:
import pandas as pd
import plotly.express as px
from unidecode import unidecode
import re
import numpy as np
from os import listdir
from collections import Counter

In [2]:
PATH_RAW = r'C:\Users\heylu\Documents\github\Sentimental Analysis -  A Study Case About Mental Health in Pandemic Times\data\raw'
PATH_PROCESSED = r'C:\Users\heylu\Documents\github\Sentimental Analysis -  A Study Case About Mental Health in Pandemic Times\data\processed'

In [3]:
def cleaning_tweets(data, language='pt'):
    dataset = data.copy()
    dataset = dataset[dataset['language'] == language]
    dataset.drop_duplicates('tweet', inplace=True)
    dataset.drop_duplicates('username', inplace=True)
    dataset['hashtags'] = [", ".join(y for y in re.findall('\B#\w\w+', x)) for x in dataset['tweet']]
    dataset = dataset[['date','tweet','hashtags']]
    dataset

    return dataset

In [4]:
for f in listdir(PATH_RAW):
    dataset = pd.read_csv(PATH_RAW + "\\" + f)
    dataset = cleaning_tweets(dataset)
    dataset.to_csv(PATH_PROCESSED + "\\" + f)

In [5]:
dataset = pd.DataFrame()

for f in listdir(PATH_PROCESSED):
    df = pd.read_csv(PATH_PROCESSED + "\\" + f)
    dataset = pd.concat([dataset, df])
    
dataset = dataset.drop_duplicates('tweet')

In [6]:
dataset['hashtags'] = dataset['hashtags'].fillna('#nan')
dataset['hashtags'] = dataset['hashtags'].apply(lambda x: re.sub("#", '', x))
dataset['hashtags'] = dataset['hashtags'].apply(lambda x: x.lower())
dataset['hashtags'] = [unidecode(x) for x in dataset['hashtags']]

In [7]:
dataset['hashtags'] = [x.split(',') for x in dataset['hashtags']]

In [8]:
arr = []
for x in dataset['hashtags']:
    arr.extend(x)

In [9]:
arr = [re.sub(" ","", x) for x in arr]

In [10]:
arr = pd.DataFrame(arr)

In [13]:
arr.to_csv(r'C:\Users\heylu\Documents\github\Sentimental Analysis -  A Study Case About Mental Health in Pandemic Times\data\cleaned\freq.csv')

In [162]:
dataset['tweet'] = dataset['tweet'].apply(lambda x: x.lower())
dataset['tweet'] = [unidecode(x) for x in dataset['tweet']]
dataset['tweet'] = [re.sub('\B#\w\w+[ ]', "",x) for x in dataset['tweet']]
dataset['tweet'] = [re.sub('\B@\w\w+[ ]', "",x) for x in dataset['tweet']]

In [163]:
dataset = dataset[['date', 'tweet']].reset_index().drop(columns='index')

KeyError: "['date'] not in index"

In [164]:
dataset = dataset.sort_values('date').set_index('date')

KeyError: "None of ['date'] are in the columns"

In [165]:
dataset.drop_duplicates('tweet', inplace=True)

In [166]:
dataset.to_csv(r'C:\Users\heylu\Documents\github\Sentimental Analysis -  A Study Case About Mental Health in Pandemic Times\data\cleaned\data.csv')

In [173]:
dataset['qtd'] = 1

In [189]:
dataset_ts = dataset[['qtd']]

In [194]:
dataset_ts.index = pd.to_datetime(dataset_ts.index)

ValueError: Inferred frequency None from passed values does not conform to passed frequency D

In [196]:
dataset_ts

,qtd
date,
2014-03-06,1
2014-03-10,1
2014-03-11,1
2014-03-11,1
2014-03-12,1
...,...
2021-04-29,1
2021-04-29,1
2021-04-29,1


In [204]:
dataset_ts = dataset_ts.groupby('date').sum()

In [206]:
dataset_ts = dataset_ts.asfreq('D')

In [210]:
dataset_ts.to_csv(r'C:\Users\heylu\Documents\github\Sentimental Analysis -  A Study Case About Mental Health in Pandemic Times\data\cleaned\time_series_data.csv')